In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Requirements
#python==3.7.3
#!pip install torch==1.1.0
!pip install scikit-learn==0.21.3
!pip install numpy==1.17.2
#!pip install sent2vec

In [ ]:
#install sent2vec
%cd /content/drive/My\ Drive/Legal\ DS/Paheli_new_corpus/semantic-segmentation/sent2vec/
!pip install .

In [ ]:
#install sentenceBERT
!pip install -U sentence-transformers

In [ ]:
import warnings
import json
import os
import string
from collections import defaultdict
#from sentence_transformers import SentenceTransformer
import sent2vec
import numpy as np
warnings.filterwarnings('ignore')

In [ ]:
%cd /content/drive/My\ Drive/Legal\ DS/Paheli_new_corpus/semantic-segmentation/

DATASET PREP

In [ ]:
f = open('Corpus/train.json')
train_corpus = json.load(f)
f.close()
f = open('Corpus/dev.json')
dev_corpus = json.load(f)
f.close()
# f = open('Corpus/test.json')
# test_corpus = json.load(f)
# f.close()
# f = open('Corpus/sentencebert_embeddings.json')
# embedding_corpus = json.load(f)
# f.close()

In [ ]:
def create_doc (id, sentence, label, path):
  document = []
  labelled_sent = sentence + "\t" + label + "\n"
  document.append(labelled_sent)
  if(os.path.exists(f"{path}/{id}.txt")):
    f = open(f"{path}/{id}.txt", "a")
    for sent in document:
      f.write(sent)
    f.close()
  else:
    f = open(f"{path}/{id}.txt", "w")
    for sent in document:
      f.write(sent)
    f.close()

In [ ]:
def create_category (id, category, path):
  if(os.path.exists(f"{path}/{category}.txt")):
    f = open(f"{path}/{category}.txt", "a")
    f.write(f"{id} ")
    f.close()
  else:
    f = open(f"{path}/{category}.txt", "w")
    f.write(f"{id} ")
    f.close()

In [ ]:
corpus_path = 'Corpus/data_cleaned'
cat_path = 'Corpus/categories'
emb_path  = 'Corpus/pretrained_embeddings_cleaned'

In [ ]:
#Create dataset in Paheli format
for i in range(len(train_corpus)):  
  id = train_corpus[i]['id']
  category = train_corpus[i]['meta']['group']
  sentences = train_corpus[i]['annotations'][0]['result']
  #create_category(id, category, cat_path)

  for j in range(len(sentences)):
    sentence = train_corpus[i]['annotations'][0]['result'][j]['value']['text']
    sentence_cleaned = sentence.replace("\n", "")
    label = train_corpus[i]['annotations'][0]['result'][j]['value']['labels'][0]
    create_doc(id, sentence_cleaned, label, corpus_path)

In [ ]:
model = sent2vec.Sent2vecModel()
model_path = 'sent2vec.bin'
model.load_model(model_path)
emb = model.embed_sentence("once upon a time .") 

In [ ]:
print(emb)

In [ ]:
# model = sent2vec.Sent2vecModel()
# model.load_model('sent2vec.bin')
# emb = model.embed_sentence("once upon a time .") 

In [ ]:
f = open("test2.txt", "w")
for i in range(len(emb[0])):
  if(i != (len(emb[0]) -1)):
    f.write(str(emb[0][i]) + " ")
  else:
    f.write(str(emb[0][i]))  
f.close()

In [ ]:
#Create embeddings txt files
def create_embedding_files(id, sentence, label, emb_path):
  if(os.path.exists(f"{emb_path}/{id}.txt")):
    write = "a"
  else:
    write = "w"
  f = open(f"{emb_path}/{id}.txt", write)
  for i in range(len(sentence[0])):
    if(i != (len(sentence[0]) - 1)):
      f.write(str(sentence[0][i]) + " ")
    else:
      f.write(str(sentence[0][i]))
  f.write("\t" + label + "\n")  
  f.close()

In [ ]:
#Create and save embeddings
for i in range(len(train_corpus)):  
  id = train_corpus[i]['id']
  sentences = train_corpus[i]['annotations'][0]['result']

  for j in range(len(sentences)):
    sentence = train_corpus[i]['annotations'][0]['result'][j]['value']['text']
    sentence_cleaned = sentence.replace("\n", "")
    emb_sent = model.embed_sentence(sentence_cleaned) 
    label = train_corpus[i]['annotations'][0]['result'][j]['value']['labels'][0]

    #if all zeros -> label = none
    if (not np.any(emb_sent)):
      label = 'NONE'

    create_embedding_files(id, emb_sent, label, emb_path)
    

In [ ]:
#Create SentenceBERT embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
#Create SBERT embeddings txt files
def create_SBERT_embedding_files(id, sentence, label, emb_path):
  if(os.path.exists(f"{emb_path}/{id}.txt")):
    write = "a"
  else:
    write = "w"
  f = open(f"{emb_path}/{id}.txt", write)
  for i in range(len(sentence)):
    if(i != (len(sentence) - 1)):
      f.write(str(sentence[i]) + " ")
    else:
      f.write(str(sentence[i]))
  f.write("\t" + label + "\n")  
  f.close()

In [ ]:
#Create and save SBERT embeddings

emb_path = 'Corpus/pretrained_sentencebert'

for i in range(len(train_corpus)):  
  id = train_corpus[i]['id']
  sentences = train_corpus[i]['annotations'][0]['result']

  for j in range(len(sentences)):
    sentence = train_corpus[i]['annotations'][0]['result'][j]['value']['text']
    sentence_cleaned = sentence.replace("\n", "")
    emb_sent = model.encode(sentence_cleaned) 
    label = train_corpus[i]['annotations'][0]['result'][j]['value']['labels'][0]

    #if all zeros -> label = none
    if (not np.any(emb_sent)):
      label = 'NONE'

    create_SBERT_embedding_files(id, emb_sent, label, emb_path)
    

In [ ]:
#SBERT
!python run.py --pretrained True --data_path 'Corpus/pretrained_sentencebert/' --cat_path 'Corpus/categories/categories.txt' --dataset_size 245

RUN

In [ ]:
!python run.py --pretrained True --data_path 'Corpus/pretrained_embeddings/' --cat_path 'Corpus/categories/categories.txt' --dataset_size 245

In [ ]:
!python run.py --pretrained True --data_path 'Corpus/pretrained_embeddings/' --cat_path 'Corpus/categories/categories.txt' --dataset_size 245

In [ ]:
#Cleaned
!python run.py --data_path 'Corpus/data_cleaned/' --cat_path 'Corpus/categories/categories.txt' --dataset_size 245

In [ ]:
!python run.py --pretrained True --data_path 'Corpus/pretrained_embeddings_cleaned/' --cat_path 'Corpus/categories/categories.txt' --dataset_size 245

In [ ]:
train_corpus[0]

In [ ]:
####Pre Ekstep

In [ ]:
model = sent2vec.Sent2vecModel()
model_path = 'sent2vec.bin'
model.load_model(model_path)

In [ ]:
f = open('Corpus/train.json')
train_corpus = json.load(f)
f.close()
f = open('Corpus/dev.json')
dev_corpus = json.load(f)
f.close()

In [ ]:
#Create and save embeddings
for i in range(len(train_corpus)):  
  sentences = train_corpus[i]['annotations'][0]['result']

  for j in range(len(sentences)):
    sentence = train_corpus[i]['annotations'][0]['result'][j]['value']['text']
    sentence_cleaned = sentence.replace("\n", "")
    emb_sent = model.embed_sentence(sentence_cleaned)
    array_2_list = list(emb_sent[0])
    list_2_str = ' '.join(str(e) for e in array_2_list)

    train_corpus[i]['annotations'][0]['result'][j]['value']['text'] = list_2_str



In [ ]:
#Create and save embeddings
for i in range(len(dev_corpus)):  
  sentences = dev_corpus[i]['annotations'][0]['result']

  for j in range(len(sentences)):
    sentence = dev_corpus[i]['annotations'][0]['result'][j]['value']['text']
    sentence_cleaned = sentence.replace("\n", "")
    emb_sent = model.embed_sentence(sentence_cleaned)
    array_2_list = list(emb_sent[0])
    list_2_str = ' '.join(str(e) for e in array_2_list)

    dev_corpus[i]['annotations'][0]['result'][j]['value']['text'] = list_2_str 



In [ ]:
with open('pre_train_ekstep.json', 'w') as jsonfile:
   json.dump(train_corpus, jsonfile)

In [ ]:
with open('pre_dev_ekstep.json', 'w') as jsonfile:
   json.dump(dev_corpus, jsonfile)

In [ ]:
test1 = train_corpus[0]['annotations'][0]['result'][0]['value']['text']
print(test1)

In [ ]:
sentence_cleaned = test1.replace("\n", "")
emb_sent = model.embed_sentence(sentence_cleaned)

In [ ]:
emb_sent[0]

In [ ]:
test2 = list(emb_sent[0])
len(test2)
list_2_str = ''.join(str(e) for e in test2)
list_2_str

In [ ]:
test = np.array2string(emb_sent[0], formatter = {'float_kind':lambda x: "%.7f" % x} )
test.replace("\n", "")

In [ ]:
test[1]